# Micki Rhen HW1 - Classification models in sklearn

The goal of homework 1 is to build a classifier model to predict political party using other variables

## Preliminaries

In [4]:
# To auto-reload modules in jupyter notebook (so that changes in files *.py doesn't require manual reloading):
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Import commonly used libraries and magic command for inline plotting

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [6]:
%matplotlib inline

## The Initial Tasks

### Task 1 - Create project folder structure

Task 1 involved creating a new project folder structure with the cookiecutter-datascience-simple template which was introduced in Module 1.


The following script was run from the directory of this new project 'rhenhw1':

cookiecutter https://github.com/misken/cookiecutter-datascience-simple

![check](images/check.PNG)

### Task 2 - Put the new project folder under version control using git

After moving the supplied data to ...\data\raw, the initial version control was implemented using the following commands:
    
    * git add *.ipynb
    * git add docs/*.md
    * git add .gitignore
    * git commit -m 'initial commit'
    
A new repository was created online for rhenhw1 and GitHub desktop was used to push the committed files to this repository

Adds and commits should be done periodically to backup work as it is being produced.  Other types of files (.py, etc) may be added at these later times

![check](images/check.PNG)

## The Real Work

Build at least one logistic regression model (with regularization) and one random forest model to predict PoliticalParty.